#### Imports

In [1]:
import os
import json
from tqdm import tqdm

from configs import ConfigPath
from data_preprocessing.text_splitter import TextSplitter
from utils.utils import read_json_file
from llms.embedding_model import EmbeddingModel
from knowledge_graph.loader import GraphLoader
from knowledge_graph.crud import GraphCrud
from configs.config import ConfigEnv

#### Initializations

In [2]:
data = read_json_file(file_path=os.path.join(ConfigPath.RAW_DATA_DIR, "pqa_labeled.json"))  

text_splitter = TextSplitter()
embedding_model = EmbeddingModel()
crud = GraphCrud(uri=ConfigEnv.NEO4J_URI, 
                 user=ConfigEnv.NEO4J_USER,
                 password=ConfigEnv.NEO4J_PASSWORD,
                 database=ConfigEnv.NEO4J_DB)

graph_loader = GraphLoader(text_splitter=text_splitter,
                           embedding_model=embedding_model,
                           crud=crud,
                           data=data)



2025-02-26 21:38:19,388 [DEBUG] embedding_model - CUDA is available, using GPU
2025-02-26 21:38:38,725 [DEBUG] embedding_model - Embedding model initialized: neuml/pubmedbert-base-embeddings


Connection successful!


In [ ]:
# data = read_json_file(file_path=os.path.join(ConfigPath.RAW_DATA_DIR, "pqa_labeled.json"))       

In [3]:
data_for_load = graph_loader.prepare_data_for_load()

100%|██████████| 1000/1000 [00:59<00:00, 16.78it/s]


In [ ]:
# graph_loader.load_data_to_graph()

  0%|          | 0/1000 [00:00<?, ?it/s]2025-02-26 21:40:24,973 [DEBUG] crud - Created node with label 'ARTICLE' and elementId: 4:31b5014d-cb89-451c-ae23-47ad943d4cb4:3408
2025-02-26 21:40:24,981 [DEBUG] crud - Created node with label 'CONTEXT' and elementId: 4:31b5014d-cb89-451c-ae23-47ad943d4cb4:3409
2025-02-26 21:40:25,014 [DEBUG] crud - Created relationship `HAS_CONTEXT` from node 4:31b5014d-cb89-451c-ae23-47ad943d4cb4:3408 to 4:31b5014d-cb89-451c-ae23-47ad943d4cb4:3409
2025-02-26 21:40:25,024 [DEBUG] crud - Created relationship `HAS_MESH_TERM` from node 4:31b5014d-cb89-451c-ae23-47ad943d4cb4:3409 to Alismataceae
2025-02-26 21:40:25,032 [DEBUG] crud - Created relationship `HAS_MESH_TERM` from node 4:31b5014d-cb89-451c-ae23-47ad943d4cb4:3409 to Apoptosis
2025-02-26 21:40:25,040 [DEBUG] crud - Created relationship `HAS_MESH_TERM` from node 4:31b5014d-cb89-451c-ae23-47ad943d4cb4:3409 to Cell Differentiation
2025-02-26 21:40:25,046 [DEBUG] crud - Created relationship `HAS_MESH_TERM` fr

### Nodes deletion

In [8]:
# graph_loader.crud.delete_node(node_id="4:31b5014d-cb89-451c-ae23-47ad943d4cb4:3409")